In [7]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [8]:
client = AzureOpenAIChatCompletionClient(
    model=os.environ["AZURE_OPENAI_MODEL"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"]
)

In [9]:
planner_agent = AssistantAgent(
    name="planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips",
    system_message="""
    You are a Front Desk Travel Agent with experiences to deal with many customers.
    Your goal is to provide the best activities and locations for a traveler to visit.
    You provide a single recommendation per person.
    Don't waste time with chit chat.
    """
)
concierge_agent = AssistantAgent(
    name="concierge_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit",
    system_message=""" 
    You are a hotel concierge. You provide the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, respond with 'APPROVE'
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """
)

In [12]:
termination = TextMentionTermination("APPROVE") # conditions for the termination of the conversation.
team = RoundRobinGroupChat( # create a discussion 'team' consisting of 2 agents
    [planner_agent, concierge_agent], termination_condition=termination
)

async for message in team.run_stream(task="I would like to plan a trip to Paris."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(f"{message.source}: {message.content}")

user: I would like to plan a trip to Paris.
planner_agent: I recommend visiting the Palais-Royal and its beautiful gardens. It's a peaceful spot filled with history, art, and unique shops. Enjoy a leisurely stroll through the gardens and appreciate the lovely architecture surrounding the courtyard.
concierge_agent: APPROVE
Stop Reason: Text 'APPROVE' mentioned
